### Testing Poly Websocket

In [ ]:
from polygon import WebSocketClient
from polygon.websocket.models import WebSocketMessage
from typing import List

API_KEY = "s2zC3SQEf3MgBQzTygqvYc9QfLNI_ABq"

ws = WebSocketClient(api_key=API_KEY,market="crypto", subscriptions=["X:BTCUSD"])


def handle_msg(msg: List[WebSocketMessage]):
    for m in msg:
        print(m)


ws.run(handle_msg=handle_msg)

ws.close

In [ ]:
from polygon import WebSocketClient
from polygon.websocket.models import WebSocketMessage, CryptoTrade
from typing import List
import asyncio

async def handle_msg(msg: List[WebSocketMessage]):
    for m in msg:
        if isinstance(m, CryptoTrade):
            await print(m)

# Function to run the WebSocket client
async def run_ws():
    await ws.connect(processor=handle_msg)

# Main function to run the event loop
async def main():
    await asyncio.gather(
        run_ws()
    )

# Standard boilerplate to run the main function
if __name__ == "__main__":
    asyncio.run(main())


from polygon import WebSocketClient
from polygon.websocket.models import WebSocketMessage
from typing import List

API_KEY = "s2zC3SQEf3MgBQzTygqvYc9QfLNI_ABq"

ws = WebSocketClient(api_key=API_KEY,market="crypto", subscriptions=["X:BTCUSD"])

ws.run(handle_msg=handle_msg)

ws.close



In [ ]:
import nest_asyncio
import asyncio
from polygon import WebSocketClient
from polygon.websocket.models import WebSocketMessage, CryptoTrade
from typing import List
import logging

from TrendAnalysis import process_trade

API_KEY = "s2zC3SQEf3MgBQzTygqvYc9QfLNI_ABq"

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

ws = WebSocketClient(api_key=API_KEY, subscriptions=["X:BTCUSD"])

logger.info("Trading will commence now")

# Function to handle incoming messages
async def handle_msg(msg: List[WebSocketMessage]):
    for m in msg:
        logger.info(m)
    '''
    for m in msg:
        if isinstance(m, CryptoTrade):
            await process_trade(m)
    '''

# Function to run the WebSocket client
async def run_ws():
    try:
        await ws.connect(processor=handle_msg)
    except Exception as e:
        logger.error(f"WebSocket connection error: {e}")

# Main function to run the event loop
async def main():
    await asyncio.gather(run_ws())

# Standard boilerplate to run the main function
if __name__ == "__main__":
    # Apply nest_asyncio to allow nested event loops
    nest_asyncio.apply()
    asyncio.run(main())

### Poly Bars API

In [ ]:
import requests
import datetime

# Replace with your Polygon.io API key
api_key = "s2zC3SQEf3MgBQzTygqvYc9QfLNI_ABq"

# Set your parameters
crypto_ticker = "X:BTCUSD"
multiplier = 1
timespan = "second"
from_date = "2024-01-01"
to_date = datetime.datetime.now().strftime('%Y-%m-%d')

# Build the request URL
url = f"https://api.polygon.io/v2/aggs/ticker/{crypto_ticker}/range/{multiplier}/{timespan}/{from_date}/{to_date}"

# Add your API key to the request parameters
params = {
    "apiKey": api_key
}

# Make the request
response = requests.get(url, params=params)
data = response.json()

# Check if the request was successful
if response.status_code == 200:
    print(data)
else:
    print(f"Error: {data['error']}")

# Print the retrieved data
for bar in data['results']:
    print(bar)

### Benchmarking Class Types for Trade Data Buffer

In [ ]:
import timeit
import pandas as pd

# Global variable setup
BUFFER_SIZE = 15
buffer_global = [None] * BUFFER_SIZE

def append_to_global(data):
    buffer_global.append(data)
    buffer_global.pop(0)

# Class variable setup
class CircularBuffer:
    def __init__(self, size):
        self.size = size
        self.buffer = pd.DataFrame(index=range(size), columns=["timestamp", "price", "volume"])  # Adjust columns as needed
        self.index = 0

    def append(self, data):
        # Convert trade to pandas Series
        trade_series = pd.Series(data)
        # Place new trade in the current index position
        self.buffer.iloc[self.index] = trade_series
        # Update index in a circular manner
        self.index = (self.index + 1) % self.size

class OP_CircularBuffer:
    def __init__(self, size):
        self.size = size
        self.buffer = pd.DataFrame(columns=["timestamp", "price", "volume"])  # Adjust columns as needed

    def append(self, data):
        # Convert trade to pandas Series
        trade_series = pd.Series(data, index=self.buffer.columns)
        if len(self.buffer) == 15:
            self.buffer = self.buffer.shift(-1)
            self.buffer.iloc[-1] = trade_series
        elif len(self.buffer) < 15:
            self.buffer = pd.concat([self.buffer, trade_series.to_frame().T], ignore_index=True)
        else:
            print(f"Tade Buffer Error - Buffer size {len(self.buffer)}")


buffer_class = CircularBuffer(size=15)
OPbuff_class = OP_CircularBuffer(size=15)

# Benchmark global variable
def benchmark_global():
    for i in range(100):
        append_to_global(i)

# Benchmark class variable
def benchmark_class():
    for i in range(1):
        i = {"timestamp": 1, "price":2, "volume":3}
        buffer_class.append(i)

def OPbenchmark_class():
    for i in range(1):
        i = {"timestamp": 1, "price":2, "volume":3}
        OPbuff_class.append(i)

# Run benchmarks
time_global = timeit.timeit(benchmark_global, number=200)
print(f"Global variable time: {time_global}")
time_class = timeit.timeit(benchmark_class, number=200)
print(f"Class variable time: {time_class}")
OPtime_class = timeit.timeit(OPbenchmark_class, number=200)
print(f"OP Class variable time: {OPtime_class}")


### Feather vs Parquet for Data dumping

In [ ]:
import pandas as pd
import pyarrow.feather as feather
import timeit

# Function to write data to Feather
def write_to_feather(data, filename='data.feather'):
    df = pd.DataFrame([data])
    feather.write_feather(df, filename)

# Function to write data to Parquet
def write_to_parquet(data, filename='data.parquet'):
    df = pd.DataFrame([data])
    df.to_parquet(filename, index=False)

# Function to append data repeatedly to Feather
def benchmark_feather():
    data = {"timestamp": pd.Timestamp.now(), "price": 100, "volume": 10}
    for _ in range(2):
        write_to_feather(data)

# Function to append data repeatedly to Parquet
def benchmark_parquet():
    data = {"timestamp": pd.Timestamp.now(), "price": 100, "volume": 10}
    for _ in range(2):
        write_to_parquet(data)

# Run benchmarks
time_feather = timeit.timeit(benchmark_feather, number=1000)
time_parquet = timeit.timeit(benchmark_parquet, number=1000)

print(f"Time for Feather: {time_feather}")
print(f"Time for Parquet: {time_parquet}")




In [ ]:
import pandas as pd
import pyarrow.feather as feather

# Path to the Feather file
file_path = 'data.feather'

# Read the Feather file into a Pandas DataFrame
df = feather.read_feather(file_path)

# Print the DataFrame
print(df.info())


In [ ]:
import pandas as pd
import ast

with open('data.txt', 'r') as file:
    content = file.read()

Dict_list = ast.literal_eval(content)

df = pd.DataFrame(Dict_list)

print(df.info())
print(df.head())